# MLIP and Espresso
In this tutorial, you'll learn how to train and deploy a machine-learned interatomic potential (MLIP) with espresso. 

In [1]:
import zntrack

In [2]:
model = zntrack.from_rev("r_max_6_Apax")

/work/fzills/projects/2025-espresso-water/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import rdkit2ase

In [4]:
water = rdkit2ase.smiles2conformers("O", numConfs=100)

In [5]:
box = rdkit2ase.pack([water], counts=[100], density=997, packmol="/tikhome/fzills/.local/bin/packmol")

In [6]:
calc = model.get_calculator()

/work/fzills/projects/2025-espresso-water/.venv/lib/python3.11/site-packages/zntrack/state.py:159: UserWarning: The temporary path is not used when neither remote or rev are set.Consider checking for `self.state.remote` and `self.state.rev` whenusing `with node.state.use_tmp_path(): ...` .
  warnings.warn(


In [7]:
box.calc = calc

In [8]:
box.get_potential_energy()

-46855.0459166456

In [9]:
b2 = box.copy()

In [10]:
b2.calc = calc

In [11]:
b2.get_potential_energy()

-46855.0459166456

In [12]:
results = calc.batch_eval([box])
results[0].calc.results

{'energy': -46855.04023576579,
 'energy_ensemble': array([-46858.71355196, -46854.51146257, -46843.02679578, -46843.304312  ,
        -46856.54263406, -46858.03521649, -46863.12152852, -46851.38268893,
        -46855.23988012, -46863.03724384, -46855.31525073, -46862.13718659,
        -46852.53403051, -46853.97067579, -46863.54628076, -46846.22503361]),
 'energy_uncertainty': array(6.62147771),
 'forces': array([[ 1.19635487e+00, -1.46638572e+00,  3.17794309e+00],
        [ 1.43354927e+00,  9.32345191e-01,  5.07379803e-03],
        [-1.72685401e+00, -3.56580535e-01,  1.49837700e-01],
        [ 3.55410314e-01, -1.63150916e-01,  1.25290843e+00],
        [-4.08918936e-01,  1.11056225e+00, -2.07224115e-01],
        [ 3.31315890e-01, -1.87356687e+00, -1.10426699e+00],
        [-1.13255367e+00, -1.88139242e+00, -1.48517753e-01],
        [ 1.52520228e+00, -1.02227363e-01,  3.18715403e-01],
        [-4.85032062e-01,  1.51590530e+00,  2.50592119e-01],
        [ 4.53970421e-01,  4.43663676e-01, 